# Evaluation of Neighborhoods in Toronto for a new Burmese Restaurant

## Capstone Project of Laura Willis

In this notebook, I will be creating clusters to find the most suitable location to open an authentic Burmese restaurant in Toronto, Canada.

In [1]:
#importing necessary libraries
import requests
import pandas as pd

In [2]:
#scrapping neighborhoods in Canada
url  = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
if page.status_code == 200:
    print('Page download successful')
else:
    print('Page download error. Error code: {}'.format(page.status_code))

Page download successful


In [3]:
df_html = pd.read_html(url, header=0, na_values = ['Not assigned'])[0]
df_html.head()

,Postal Code,Borough,Neighborhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [4]:
#Drop the the rows on which the Borough is empty
df_html.dropna(subset=['Borough'], inplace=True)

In [5]:
df_html.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [6]:
#Check Neighborhood is empty but Borough exists
n_empty_neighborhood = df_html[df_html['Neighborhood'].isna()].shape[0]
print('Number of rows on which Neighborhood column is empty: {}'.format(n_empty_neighborhood))

Number of rows on which Neighborhood column is empty: 0


In [9]:
#Group by Postcode / Borough
df_postalcodes = df_html.groupby(['Postal Code','Borough']).Neighborhood.agg([('Neighborhood', ', '.join)])
df_postalcodes.reset_index(inplace=True)
df_postalcodes.head(5)

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [10]:
#Drop the the rows on which the Borough is empty
df_html.dropna(subset=['Borough'], inplace=True)

In [13]:
# Check Neighborhood is empty but Borough exists
n_empty_neighborhood = df_html[df_html['Neighborhood'].isna()].shape[0]
print('Number of rows on which Neighborhood column is empty: {}'.format(n_empty_neighborhood))

Number of rows on which Neighborhood column is empty: 0


In [15]:
print('The shape of the dataset is:',df_postalcodes.shape)

The shape of the dataset is: (103, 3)


In [24]:
#to make it easier, we will store this in csv format.
#Export to .CSV
df_postalcodes.to_csv('Toronto_Postalcodes.csv')

In [35]:
import numpy as np

I am having issues with the GeoEncoder library, so I will use the csv file downloaded instead

In [36]:
#Read CSV file from link and load into dataframe
url_csv = 'http://cocl.us/Geospatial_data'
df_coordinates = pd.read_csv(url_csv)
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [37]:
#use the previously cleaned data
df_neighborhoods = pd.read_csv('Toronto_Postalcodes.csv',index_col=[0])
df_neighborhoods.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [40]:
# Make sure both dataframes have the same naming convention
df_coordinates.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
df_neighborhoods.rename(columns={'Postcode': 'PostalCode'}, inplace=True)

In [41]:
#check both databases separately. First Neighborhoods
df_neighborhoods.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [42]:
#now the coordinates database
df_coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [51]:
#fix it so both naming conventions for Post code are the same so we can merge the tables
df_neighborhoods.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

In [52]:
df_neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [53]:
# Merge both datasets
df_neighborhoods_coordinates = pd.merge(df_neighborhoods, df_coordinates, on='PostalCode')
df_neighborhoods_coordinates.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [54]:
# Check coordinates for a few of neighborhoods
df_neighborhoods_coordinates[(df_neighborhoods_coordinates['PostalCode']=='M5G') |
                             (df_neighborhoods_coordinates['PostalCode']=='M2H') ]

,PostalCode,Borough,Neighborhood,Latitude,Longitude
17,M2H,North York,Hillcrest Village,43.803762,-79.363452
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [55]:
#Export merged dataframe to .CSV
df_neighborhoods_coordinates.to_csv('Toronto_Postalcodes_2.csv')

In [63]:
#having trouble with folium. Going to troubleshoot for a few lines
!pip -q install folium
import folium 

#hoping that worked!

In [64]:
from sklearn.cluster import KMeans

In [65]:
# Read .csv file from above
df = pd.read_csv('Toronto_Postalcodes_2.csv', index_col=0)
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [66]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [67]:
#count Bourough and Neighborhood
df.groupby('Borough').count()['Neighborhood']

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
East York            5
Etobicoke           12
Mississauga          1
North York          24
Scarborough         17
West Toronto         6
York                 5
Name: Neighborhood, dtype: int64

In [68]:
df_toronto = df[df['Borough'].str.contains('Toronto')]
df_toronto.reset_index(inplace=True)
df_toronto.drop('index', axis=1, inplace=True)
df_toronto.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [69]:
#Check the number of neighborhoods
print(df_toronto.groupby('Borough').count()['Neighborhood'])

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
West Toronto         6
Name: Neighborhood, dtype: int64


In [70]:
#Create list with the Boroughs (to be used later)
boroughs = df_toronto['Borough'].unique().tolist()

In [71]:
#Obtain the coordinates from the dataset itself, just averaging Latitude/Longitude of the current dataset 
lat_toronto = df_toronto['Latitude'].mean()
lon_toronto = df_toronto['Longitude'].mean()
print('The geographical coordinates of Toronto are {}, {}'.format(lat_toronto, lon_toronto))

The geographical coordinates of Toronto are 43.66713498717948, -79.38987324871795


In [72]:
borough_color = {}
for borough in boroughs:
    borough_color[borough]= '#%02X%02X%02X' % tuple(np.random.choice(range(256), size=3)) #Random color

In [73]:
map_toronto = folium.Map(location=[lat_toronto, lon_toronto], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], 
                                           df_toronto['Longitude'],
                                           df_toronto['Borough'], 
                                           df_toronto['Neighborhood']):
    label_text = borough + ' - ' + neighborhood
    label = folium.Popup(label_text)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=borough_color[borough],
        fill_color=borough_color[borough],
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In case you are unable to see the above on github, it is a map of Toronto with dots in green, pink and brown, with each color of dots clustered together.

### Getting Venues Data using Foursquare

In [75]:
CLIENT_ID = 'PZXIOF0W1I0CPDBOQTHEBFDULHAH34DAHVF2DMHWS2AKQE0K' # your Foursquare ID
CLIENT_SECRET = 'GGCXFWHGKTJZAKLRR5BQOE2PS4QERLHZLHTREEML2NT42U25' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [76]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [77]:
#Get venues for all neighborhoods in our dataset
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                latitudes=df_toronto['Latitude'],
                                longitudes=df_toronto['Longitude'])

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High 

In [78]:
#Check size of resulting dataframe
toronto_venues.shape

(1627, 7)

In [79]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Domino's Pizza,43.679058,-79.297382,Pizza Place
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


In [80]:
#Let's see the number of venues per neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",17,17,17,17,17,17
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,66,66,66,66,66,66
Christie,17,17,17,17,17,17
Church and Wellesley,83,83,83,83,83,83
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,33,33,33,33,33,33


In [81]:
#Ok, how about the number of unique venue categories
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 233 uniques categories.


In [82]:
#Interesting. Let's look at the list of categories
toronto_venues['Venue Category'].unique()[:100]

array(['Trail', 'Health Food Store', 'Pub', 'Pizza Place', 'Neighborhood',
       'Asian Restaurant', 'Cosmetics Shop', 'Greek Restaurant',
       'Ice Cream Shop', 'Italian Restaurant', 'Brewery',
       'Fruit & Vegetable Store', 'Yoga Studio', 'Restaurant',
       'Juice Bar', 'Bookstore', 'Bubble Tea Shop', 'Dessert Shop',
       'Furniture / Home Store', 'Spa', 'Grocery Store', 'Coffee Shop',
       'Bakery', 'Caribbean Restaurant', 'Japanese Restaurant',
       'Indian Restaurant', 'Café', 'Lounge', 'Frozen Yogurt Shop',
       'Liquor Store', 'American Restaurant', 'Gym', 'Fish & Chips Shop',
       'Fast Food Restaurant', 'Sushi Restaurant', 'Park', 'Pet Store',
       'Steakhouse', 'Burrito Place', 'Movie Theater', 'Sandwich Place',
       'Food & Drink Shop', 'Fish Market', 'Gay Bar', 'Thai Restaurant',
       'Seafood Restaurant', 'Cheese Shop', 'Comfort Food Restaurant',
       'Middle Eastern Restaurant', 'Stationery Store', 'Coworking Space',
       'Wine Bar', 'Latin Ame

In [83]:
# check if the results contain "Asian Restaurants"
"Asian Restaurant" in toronto_venues['Venue Category'].unique()

True

In [85]:
# check if the results contain "Thai Restaurants"
#please note I changed the data to Thai because I was previously writing the code using Asian but the number is so small
"Thai Restaurant" in toronto_venues['Venue Category'].unique()

True

### Analyze each Neighborhood

In [86]:
# one hot encoding
to_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
to_onehot['Neighborhoods'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

print(to_onehot.shape)
to_onehot.head()

(1627, 234)


,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [87]:
to_grouped = to_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(to_grouped.shape)
to_grouped

(39, 234)


,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667
2,"Business reply mail Processing Centre, South C...",0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0625,0.0625,0.0625,0.0625,0.1875,0.125,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.015152,0.000000,0.000000,0.015152,0.000000,0.000000,0.015152
5,Christie,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.012048,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.012048,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.012048,0.000000,0.024096
7,"Commerce Court, Victoria Hotel",0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.040000,0.000000,...,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000
8,Davisville,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.000000,0.000000,...,0.060606,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [88]:
len(to_grouped[to_grouped["Thai Restaurant"] > 0])

12

Create a new dataframe to find Thai Restaurants only

In [89]:
to_thai = to_grouped[["Neighborhoods","Thai Restaurant"]]

In [90]:
to_thai.head()

,Neighborhoods,Thai Restaurant
0,Berczy Park,0.017241
1,"Brockton, Parkdale Village, Exhibition Place",0.000000
2,"Business reply mail Processing Centre, South C...",0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000
4,Central Bay Street,0.030303


### Cluster Neighborhoods

Let's now run k-means to cluster the neighborhoods in Toronto into 3 clusters.

In [91]:
# set number of clusters
toclusters = 3

to_clustering = to_thai.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=toclusters, random_state=0).fit(to_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 1, 1, 1, 0, 1, 1, 0, 0, 1], dtype=int32)

In [92]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
to_merged = to_thai.copy()

# add clustering labels
to_merged["Cluster Labels"] = kmeans.labels_

In [93]:
to_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
to_merged.head()

,Neighborhood,Thai Restaurant,Cluster Labels
0,Berczy Park,0.017241,0
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,1
2,"Business reply mail Processing Centre, South C...",0.000000,1
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,1
4,Central Bay Street,0.030303,0


In [94]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
to_merged = to_merged.join(toronto_venues.set_index("Neighborhood"), on="Neighborhood")

print(to_merged.shape)
to_merged.head()

(1627, 9)


,Neighborhood,Thai Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,0.017241,0,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store
0,Berczy Park,0.017241,0,43.644771,-79.373306,The Keg Steakhouse + Bar - Esplanade,43.646712,-79.374768,Restaurant
0,Berczy Park,0.017241,0,43.644771,-79.373306,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant
0,Berczy Park,0.017241,0,43.644771,-79.373306,Meridian Hall,43.646292,-79.376022,Concert Hall
0,Berczy Park,0.017241,0,43.644771,-79.373306,Hockey Hall Of Fame (Hockey Hall of Fame),43.646974,-79.377323,Museum


In [95]:
# sort the results by Cluster Labels
print(to_merged.shape)
to_merged.sort_values(["Cluster Labels"], inplace=True)
to_merged

(1627, 9)


,Neighborhood,Thai Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,0.017241,0,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store
11,"First Canadian Place, Underground city",0.020000,0,43.648429,-79.382280,Astarté Fresh Yogurt Bar,43.647596,-79.386419,Café
11,"First Canadian Place, Underground city",0.020000,0,43.648429,-79.382280,Cosmopolitan Toronto Centre Hotel & Spa,43.649064,-79.377598,Hotel
32,Studio District,0.024390,0,43.659526,-79.340923,Coach House Shortbread Co.,43.662501,-79.341102,Bakery
32,Studio District,0.024390,0,43.659526,-79.340923,McQueens Pub,43.661483,-79.338072,Gastropub
32,Studio District,0.024390,0,43.659526,-79.340923,Saulter Street Brewery,43.658412,-79.346392,Brewery
32,Studio District,0.024390,0,43.659526,-79.340923,Good Neighbour,43.661000,-79.340298,Clothing Store
32,Studio District,0.024390,0,43.659526,-79.340923,Gale's Snack Bar,43.658239,-79.339077,Diner
32,Studio District,0.024390,0,43.659526,-79.340923,TD Canada Trust,43.660659,-79.342775,Bank
32,Studio District,0.024390,0,43.659526,-79.340923,7-Eleven,43.660897,-79.340159,Convenience Store


Time to see what the clusters look like

In [96]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [97]:
# create map
map_clusters = folium.Map(location=[lat_toronto, lon_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(toclusters)
ys = [i+x+(i*x)**2 for i in range(toclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(to_merged['Neighborhood Latitude'], to_merged['Neighborhood Longitude'], to_merged['Neighborhood'], to_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster))
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [98]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### Let's examine the clusters

In [99]:
#Cluster 0
to_merged.loc[to_merged['Cluster Labels'] == 0]

,Neighborhood,Thai Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,0.017241,0,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store
11,"First Canadian Place, Underground city",0.020000,0,43.648429,-79.382280,Astarté Fresh Yogurt Bar,43.647596,-79.386419,Café
11,"First Canadian Place, Underground city",0.020000,0,43.648429,-79.382280,Cosmopolitan Toronto Centre Hotel & Spa,43.649064,-79.377598,Hotel
32,Studio District,0.024390,0,43.659526,-79.340923,Coach House Shortbread Co.,43.662501,-79.341102,Bakery
32,Studio District,0.024390,0,43.659526,-79.340923,McQueens Pub,43.661483,-79.338072,Gastropub
32,Studio District,0.024390,0,43.659526,-79.340923,Saulter Street Brewery,43.658412,-79.346392,Brewery
32,Studio District,0.024390,0,43.659526,-79.340923,Good Neighbour,43.661000,-79.340298,Clothing Store
32,Studio District,0.024390,0,43.659526,-79.340923,Gale's Snack Bar,43.658239,-79.339077,Diner
32,Studio District,0.024390,0,43.659526,-79.340923,TD Canada Trust,43.660659,-79.342775,Bank
32,Studio District,0.024390,0,43.659526,-79.340923,7-Eleven,43.660897,-79.340159,Convenience Store


In [100]:
#Cluster 1
to_merged.loc[to_merged['Cluster Labels'] == 1]

,Neighborhood,Thai Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
37,"Toronto Dominion Centre, Design Exchange",0.0,1,43.647177,-79.381576,Hilton,43.649946,-79.385479,Hotel
37,"Toronto Dominion Centre, Design Exchange",0.0,1,43.647177,-79.381576,Victoria's Restaurant,43.649298,-79.376431,Restaurant
37,"Toronto Dominion Centre, Design Exchange",0.0,1,43.647177,-79.381576,Starbucks,43.646213,-79.379088,Coffee Shop
37,"Toronto Dominion Centre, Design Exchange",0.0,1,43.647177,-79.381576,Ninki Izakaya,43.650228,-79.384863,Japanese Restaurant
37,"Toronto Dominion Centre, Design Exchange",0.0,1,43.647177,-79.381576,Starbucks,43.650159,-79.377793,Coffee Shop
37,"Toronto Dominion Centre, Design Exchange",0.0,1,43.647177,-79.381576,Brooks Brothers,43.646852,-79.379447,Clothing Store
37,"Toronto Dominion Centre, Design Exchange",0.0,1,43.647177,-79.381576,Sweet Lulu,43.650557,-79.381175,Asian Restaurant
37,"Toronto Dominion Centre, Design Exchange",0.0,1,43.647177,-79.381576,M Square Coffee Co,43.651218,-79.383555,Coffee Shop
24,"Regent Park, Harbourfront",0.0,1,43.654260,-79.360636,The Beer Store,43.657773,-79.357463,Beer Store
24,"Regent Park, Harbourfront",0.0,1,43.654260,-79.360636,FUEL+,43.653193,-79.355867,Coffee Shop


In [101]:
#Cluster 2
to_merged.loc[to_merged['Cluster Labels'] == 2]

,Neighborhood,Thai Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
15,"High Park, The Junction South",0.083333,2,43.661608,-79.464763,Junction Grill,43.665210,-79.468461,Diner
15,"High Park, The Junction South",0.083333,2,43.661608,-79.464763,See-Scape,43.665400,-79.463995,Restaurant
15,"High Park, The Junction South",0.083333,2,43.661608,-79.464763,ROUX,43.665418,-79.462392,Cajun / Creole Restaurant
15,"High Park, The Junction South",0.083333,2,43.661608,-79.464763,Cool Hand of a Girl,43.665410,-79.462822,Café
15,"High Park, The Junction South",0.083333,2,43.661608,-79.464763,Silk,43.665291,-79.466238,Thai Restaurant
15,"High Park, The Junction South",0.083333,2,43.661608,-79.464763,Pandemonium,43.665533,-79.466931,Bookstore
15,"High Park, The Junction South",0.083333,2,43.661608,-79.464763,Isaan Der,43.665311,-79.468078,Thai Restaurant
15,"High Park, The Junction South",0.083333,2,43.661608,-79.464763,Tim & Sue's No Frills,43.664243,-79.468643,Grocery Store
15,"High Park, The Junction South",0.083333,2,43.661608,-79.464763,Playa Cabana Cantina,43.665315,-79.465548,Mexican Restaurant
15,"High Park, The Junction South",0.083333,2,43.661608,-79.464763,Chica’s Nashville Hot Chicken,43.665295,-79.464888,Fried Chicken Joint


### Observations


Most of Thai restaurants are in Cluster 2 which is around Adelaide, King, Richmond areas and lowest (close to zero) in Cluster 1 areas which are North Toronto West and Parkdale areas. There may be some opportunity to open near Chinatown, St James town as the competition seems to be low. <br>
Looking at nearby venues, it seems Cluster 1 might be a good location as there are not a lot of Asian restaurants in these areas.<br> However, our entrepreneur client wants to open his Burmese restaurant, which is a relatively new concept, in an area that also has Thai food restaurants to overcome some of the barriers people may feel to try this type of cuisine. Thus, he should look to rent a space in the Cluster 2 neighborhoods to open his establishment.